In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [22]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)
llm.invoke("What is the capital of India?")

AIMessage(content='The capital of India is New Delhi.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 14, 'total_tokens': 22, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BYRBnkiKHfktO3vXI8VIVDxL2y6gN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--da141b0d-0c7c-4d42-9334-a27592f00690-0', usage_metadata={'input_tokens': 14, 'output_tokens': 8, 'total_tokens': 22, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain_core.tools import tool

In [12]:
@tool
def add(x: float, y: float) -> float:
    """Add two numbers together."""
    return x + y

@tool
def subtract(x: float, y: float) -> float:
    """Subtract two numbers."""
    return x - y

@tool
def multiply(x: float, y: float) -> float:
    """Multiply two numbers."""
    return x * y

@tool
def exponentiate(x: float, y: float) -> float:
    """Exponentiate a number."""
    return x ** y

In [13]:
add

StructuredTool(name='add', description='Add two numbers together.', args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x0000022E3330FEB0>)

In [14]:
add.name, add.description, add.input_schema

('add', 'Add two numbers together.', langchain_core.utils.pydantic.add)

In [15]:
add.args_schema.model_json_schema()

{'description': 'Add two numbers together.',
 'properties': {'x': {'title': 'X', 'type': 'number'},
  'y': {'title': 'Y', 'type': 'number'}},
 'required': ['x', 'y'],
 'title': 'add',
 'type': 'object'}

In [16]:
import json

llm_output_string = "{\"x\": 5, \"y\": 2}"  # this is the output from the LLM
llm_output_dict = json.loads(llm_output_string)  # load as dictionary
llm_output_dict

{'x': 5, 'y': 2}

In [17]:
add.func(**llm_output_dict)

7

In [18]:
exponentiate.func(**llm_output_dict)

25

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

In [20]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # must align with MessagesPlaceholder variable_name
    return_messages=True  # to return Message objects
)

C:\Users\ajite\AppData\Local\Temp\ipykernel_6820\287820297.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [26]:
from langchain.agents import initialize_agent, AgentType, create_tool_calling_agent

tools = [add, subtract, multiply, exponentiate]

agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

In [27]:
agent.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
    "intermediate_steps": []  # agent will append it's internal steps here
})

[ToolAgentAction(tool='multiply', tool_input={'x': 10.7, 'y': 7.68}, log="\nInvoking: `multiply` with `{'x': 10.7, 'y': 7.68}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jJ4WDhT6iB7qJ5x4FcR4yVPq', 'function': {'arguments': '{"x":10.7,"y":7.68}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 130, 'total_tokens': 151, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_54eb4bd693', 'id': 'chatcmpl-BYRCQq8pKlthnLlXJwKV4DDhNq728', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--645bbc0b-6c96-4870-9a41-c25d3467cbd5-0', tool_calls=[{'name': 'multiply', 'args': {'x': 10.7, 'y': 7.68}, 'id': 'call_jJ4WDh

In [28]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True
)

In [29]:
agent_executor.invoke({
    "input": "what is 10.7 multiplied by 7.68?",
    "chat_history": memory.chat_memory.messages,
})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'x': 10.7, 'y': 7.68}`


82.1759999999999910.7 multiplied by 7.68 is approximately 82.18.

> Finished chain.


{'input': 'what is 10.7 multiplied by 7.68?',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={})],
 'output': '10.7 multiplied by 7.68 is approximately 82.18.'}

In [32]:
10.7 * 7.68

82.17599999999999

In [30]:
agent_executor.invoke({
    "input": "My name is Ajit",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Nice to meet you, Ajit! How can I assist you today?

> Finished chain.


{'input': 'My name is Ajit',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Ajit', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ajit! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, Ajit! How can I assist you today?'}

In [31]:
agent_executor.invoke({
    "input": "What is nine plus 10, minus 4 * 2, to the power of 3",
    "chat_history": memory
})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'x': 9, 'y': 10}`


19.0
Invoking: `multiply` with `{'x': 4, 'y': 2}`


8.0
Invoking: `exponentiate` with `{'x': 3, 'y': 3}`


27.0
Invoking: `subtract` with `{'x': 19, 'y': 8}`


11.0
Invoking: `subtract` with `{'x': 11, 'y': 27}`


-16.0The result of \( 9 + 10 - 4 \times 2^3 \) is \(-16\).

> Finished chain.


{'input': 'What is nine plus 10, minus 4 * 2, to the power of 3',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Ajit', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ajit! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-16\\).', additional_kwargs={}, response_metadata={})],
 'output': 'The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-16\\).'}

In [33]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Your name is Ajit.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Ajit', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ajit! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-16\\).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ajit.', additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Ajit.'}

In [35]:
agent_executor.invoke({
    "input": "What is my name",
    "chat_history": memory
})



> Entering new AgentExecutor chain...
Your name is Ajit.

> Finished chain.


{'input': 'What is my name',
 'chat_history': [HumanMessage(content='what is 10.7 multiplied by 7.68?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='10.7 multiplied by 7.68 is approximately 82.18.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='My name is Ajit', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Ajit! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is nine plus 10, minus 4 * 2, to the power of 3', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The result of \\( 9 + 10 - 4 \\times 2^3 \\) is \\(-16\\).', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Ajit.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is my name', additional_kwargs={}, response_metadata={}),
  AIMessage(cont